In [8]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} a_1 + i b_1 &  a_2 + i b_2 \\ a_3 + i b_3  & -a_1 - i b_1
\end{pmatrix}
$


In [9]:
# Define polynomial variables
@polyvar w α β #r ϕ e p
@polyvar r ϕ e p

Hˢʸᵐᵇ = [ w   0
          0   0. ]

#Hˢʸᵐᵇ = [ w           α + im* β
#          α + im* β   0.        ]

@polyvar a[1:6]
@polyvar b[1:6]

J₁ˢʸᵐᵇ = [ a[1]+im*b[1]    a[2]+im*b[2] 
           a[3]+im*b[3]   -a[1]-im*b[1] ]

J₂ˢʸᵐᵇ = [  a[4]+im*b[4]  a[5]+im*b[5] 
            a[6]+im*b[6]   -a[4]+im*b[4]  ]


Jˢʸᵐᵇ = [J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ]

function basis_series(γᵢ, data_dir)
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_data.h5", γᵢ)
    ρᵉᵍˣʸ = [ρᵉ, ρᵍ, ρˣ, ρʸ]
    return(ρᵉᵍˣʸ)
end

function generate_by_linear_composition(ρᵉᵍˣʸ, ρ₀)
    l=minimum(length.(ρᵉᵍˣʸ))
    k = hcat([vec(ρᵢ[1]) for ρᵢ in ρᵉᵍˣʸ]...)\vec(ρ₀)
    ρˡⁱⁿ = reduce(+, [k[i] * ρᵉᵍˣʸ[i][1:l] for i in 1:length(k)])
end

function g_objective(γᵢ, op_num, data_dir, train_files)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        #ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_data.h5", γᵢ)
   
        #if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
        end_train = length(tᵗʳⁿ)   
        
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, Jˢʸᵐᵇ[1: op_num] )
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective (generic function with 1 method)

In [10]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

local_rand_min (generic function with 1 method)

In [11]:
data_dir = ""
println(data_dir)

models_dir = "../MODELS/"
#tests_dir = "../TESTS/"
tests_dir = ""

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

train_files = basis_files 
#test_files = ["State_B"*string(n) for n=5:6];
test_files = dodeca_files

all_files = vcat(train_files, test_files)

24-element Vector{String}:
 "State_B1"
 "State_B2"
 "State_B3"
 "State_B4"
 "State_D1"
 "State_D2"
 "State_D3"
 "State_D4"
 "State_D5"
 "State_D6"
 ⋮
 "State_D12"
 "State_D13"
 "State_D14"
 "State_D15"
 "State_D16"
 "State_D17"
 "State_D18"
 "State_D19"
 "State_D20"

In [13]:
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]
γ = [ "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

old_data_dir = "../DATA/"
new_data_dir = ""

op_num = 2

tests_data_file_name = "POP_LARGE_GAMMAS_LME_GEN_"*string(op_num)*"_random_trn4_tst2_"*date_and_time_string * ".h5"

for γᵢ in γ

    Fsid = []
    Flin = []

    println("γ =  "*γᵢ)

    poly = g_objective(γᵢ, op_num, new_data_dir, basis_files)

    sol = local_rand_min(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    #if op_num >= 1 relaxation = subs(r^2, sol) end

    #omega = subs(w, sol)

    Jˢⁱᵈ = [DenseOperator(basis, j) for j in subs.(Jˢʸᵐᵇ[1:op_num], sol)]

    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)
    if op_num >= 2 J₂ˢⁱᵈ = subs(J₂ˢʸᵐᵇ, sol) end
    #J₃ˢⁱᵈ = subs(J₃ˢʸᵐᵇ, sol) 
    #J₄ˢⁱᵈ = subs(J₄ˢʸᵐᵇ, sol)
    #J₅ˢⁱᵈ = subs(J₅ˢʸᵐᵇ, sol)
    #J₆ˢⁱᵈ = subs(J₆ˢʸᵐᵇ, sol)


    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group   

        γ_group["omega"] = convert(Float64, omega)
        #γ_group["gamma_relaxation"] = convert(Float64, relaxation)
           
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["J1"] = convert.(ComplexF64, J₁ˢⁱᵈ)
        if op_num >= 2 γ_group["J2"] = convert.(ComplexF64, J₂ˢⁱᵈ) end
        #γ_group["J3"] = convert.(ComplexF64, J₃ˢⁱᵈ)
        #γ_group["J4"] = convert.(ComplexF64, J₄ˢⁱᵈ)
        #γ_group["J5"] = convert.(ComplexF64, J₅ˢⁱᵈ)
        #γ_group["J6"] = convert.(ComplexF64, J₆ˢⁱᵈ)
        
    end

    ρᵉᵍˣʸ = basis_series(γᵢ, new_data_dir) 
    
    for df in all_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(old_data_dir*df*"_2CUT_data.h5", γᵢ)
        ρᵉˣ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        ρˡⁱⁿ = generate_by_linear_composition(ρᵉᵍˣʸ, ρₛ[1])    
        ρˡⁱⁿ = convert(Vector{Matrix{ComplexF64}}, ρˡⁱⁿ)
        bᵗˢᵗ = LiPoSID.bloch(ρˡⁱⁿ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρˡⁱⁿ]

        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), Jˢⁱᵈ)
        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᴸᴹᴱₗᵢₙ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]  

        Fᴸᴵᴺₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵉˣ, ρᵗˢᵗ)]                      

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₗᵢₙ)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            
        end
                    
        push!(Fsid, minimum(Fᴸᴹᴱₗᵢₙ))
        push!(Flin, minimum(Fᴸᴵᴺₑₓ))
    
    end

    println()

    print("Fidelity of linearity: ")
    println(minimum(Flin))

    print("Fidelity of SID: ")
    println(minimum(Fsid))
end

println(tests_data_file_name)

γ =  25.133


BoundsError: BoundsError: attempt to access 1-element Vector{Matrix{Polynomial{true, Complex{Int64}}}} at index [1:2]